In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models


# Đường dẫn đến thư mục chứa dữ liệu huấn luyện
data_dir = '/Users/lethanhtien/Desktop/UIT/MACHINE LEARNING/CS114.O11-FinalProject/images'

# Kích thước của ảnh (48x48 pixels)
img_size = (48, 48)

# Tạo đối tượng ImageDataGenerator với tỉ lệ chia là 8:2
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Tạo dữ liệu huấn luyện từ thư mục
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Sử dụng 80% dữ liệu làm tập huấn luyện
)

# Tạo dữ liệu kiểm thử từ thư mục
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Sử dụng 20% dữ liệu làm tập kiểm thử
)

# Số lượng lớp
num_classes = len(train_generator.class_indices)

# Xây dựng mô hình CNN (giống như ví dụ trước)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.75))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))  # Số lớp đầu ra

# Compile mô hình
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Hiển thị thông tin về kiến trúc mô hình
model.summary()

# Huấn luyện mô hình và lưu thông tin vào biến history
history = model.fit(train_generator, epochs=30, validation_data=validation_generator)

# In ra thông tin đầy đủ về val_accuracy
print(history.history.keys())


Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 46, 46, 32)        896       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 23, 23, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 10, 10, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 10, 10, 64)        0         
                                    

KeyboardInterrupt: 

In [49]:
from joblib import dump
dump(model, 'cnn.joblib')

['cnn.joblib']

In [37]:
print(train_generator.class_indices)

{'angry': 0, 'fear': 1, 'happy': 2, 'neutral': 3, 'sad': 4}


In [30]:
import numpy as np
from tensorflow.keras.preprocessing import image
def test(image_path):
    # Load ảnh và chuyển về kích thước và định dạng phù hợp
    img = image.load_img(image_path, target_size=(48, 48))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Thêm một chiều để tạo batch

    # Chuẩn hóa giá trị pixel về khoảng [0, 1]
    img_array /= 255.0

    # Sử dụng mô hình để dự đoán
    predictions = model.predict(img_array)

    # Lấy nhãn có xác suất cao nhất
    predicted_label = np.argmax(predictions)
    print(predictions)


    # Hiển thị kết quả
    print(f"Predicted label: {train_generator.class_indices[predicted_label]}")

In [35]:
test("/Users/lethanhtien/Desktop/UIT/MACHINE LEARNING/CS114.O11-FinalProject/images/sad/image0000145.jpg")

1/1 [==============================] - 0s 13ms/step
[[0.19471022 0.37254104 0.00445294 0.01002804 0.4182678 ]]
Predicted label: 4


In [48]:
# Đường dẫn đến ảnh bạn muốn dự đoán
image_path = "/Users/lethanhtien/Desktop/UIT/MACHINE LEARNING/CS114.O11-FinalProject/ok.png"

# Load ảnh và chuyển về kích thước và định dạng phù hợp
img = image.load_img(image_path, target_size=(48, 48))
img_array = image.img_to_array(img)
print(img_array.shape)
img_array = np.expand_dims(img_array, axis=0)  # Thêm một chiều để tạo batch

# Chuẩn hóa giá trị pixel về khoảng [0, 1]
img_array /= 255.0

print(img_array.shape)

# Sử dụng mô hình để dự đoán
predictions = model.predict(img_array)

# Lấy nhãn có xác suất cao nhất
predicted_label = np.argmax(predictions)

# Hiển thị kết quả
print(f"Predicted label: {predicted_label}")

(48, 48, 3)
(1, 48, 48, 3)
1/1 [==============================] - 0s 11ms/step
Predicted label: 3


In [87]:
%pip install --upgrade gradio

Note: you may need to restart the kernel to use updated packages.


In [60]:
import gradio as gr
import numpy as np
from PIL import Image
from joblib import load
import dlib

model = load_model('cnn_model.h5')

emotion_labels = {
    0: "Angry",
    1: "Fear",
    2: "Happy",
    3: "Neutral",
    4: "Sad",
}

# Load dlib's pre-trained face detector
detector = dlib.get_frontal_face_detector()

def predict_emotion(image):
    # Convert Gradio Image to NumPy array
    img_array = np.array(image)



    # Convert to grayscale
    img = Image.fromarray(img_array).resize((48, 48))
    print(type(img))
    img_array = np.expand_dims(img_array, axis=0)  # Add a dimension to create a batch

    # Normalize pixel values to the range [0, 1]
    img_array /= 255.0

    # Make prediction using CNN model
    predictions = model.predict(img_array)
    predicted_label = np.argmax(predictions)

    # Detect faces using dlib
    faces = detector(img_array, 1)

    if len(faces) == 0:
        cropped_face = img_array
    else:
        # Assuming there's only one face detected, you can modify this if needed
        x, y, w, h = faces[0].left(), faces[0].top(), faces[0].width(), faces[0].height()
        cropped_face = img_array[:, y:y+h, x:x+w, :]

    print("Input image shape:", img_array.shape)
    print("Processed image shape:", cropped_face.shape)

    return emotion_labels[predicted_label], cropped_face



iface = gr.Interface(
    fn=predict_emotion,
    inputs="image",
    outputs=["text", "image"],  # Specify both text and image outputs
    title="Emotion Detection",
    description="Upload an image and predict the emotion."
)

iface.launch()



Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


<class 'PIL.Image.Image'>


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 1561, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/

In [56]:
from tensorflow.keras.models import load_model, save_model

# Save the model
save_model(model, 'cnn_model.h5')

/var/folders/7w/66wzp64d641dgfzh2ypn7_xw0000gn/T/ipykernel_35204/2662188209.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'cnn_model.h5')


In [ ]:
pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.0/242.0 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━